In [1]:
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import copy
import torch as tr

# import scripts
from src.Botorch_opt import bayesian_optimiser
from src.api_helper import api_utils
from src.api_helper import env
from datetime import datetime as dt, date as dt_date

dtype = tr.float32

IndentationError: expected an indented block (api_helper.py, line 38)

##### data

In [4]:
@api_utils.transform
def api(query):
    return env.rosenbrock(query)

x0, y0 = tr.rand(2).view(1, -1), tr.tensor([[-1]], dtype=tr.float).view(-1, 1)  
r0 = env.rosenbrock(x0)


print("init query", x0); print("init reward", r0)

NameError: name 'api_utils' is not defined

In [3]:
r0 = api(x0, r0)  

NameError: name 'api' is not defined

##### OPT

In [13]:
# hyperparameters
T = 10  # total number of iterations

# gp; includes "MA2.5", "SE", "RQ", "LR", "PO"
gp_name,gp_params = "RQ",{
                          "mode": "pro",      # "raw", "add", "pro" for GPs
                          "opt":"ADAM",        # opt for MLE; includes: quasi_newton, ADAM
                          "epochs":128,       # if chosen ADAM as opt
                          "lr":1e-1,          # learning rate for ADAM
                         }
# q-parallelism (if use analytical acq_func, q must be 1)
batch_size = 1

hyper_params = { 
    "acq_name" : "UCB",          # acqu func; includes: "EI", "UCB", "qEI", "qUCB", "qKG"
    "N_start": 32,               # number of starts for multi-start SGA
    "raw_samples" :512,          # heuristic initialisation 
    "N_MC_sample" : 256,         # number of samples for Monte Carlo simulation
    "num_fantasies": 128,        # number of fantasies used by KG
    "beta":1.,                   # used by UCB/qUCB
               }

bayes_opt = bayesian_optimiser(gp_name,gp_params,hyper_params)

In [20]:
%%time
xs, ys = bayes_opt.outer_loop(T, x0, y0, r0, api, batch_size)

Iter 128 - Loss: 0.047
acq_func took 3.8s
api took 13.6s
time step 1, drop -6.63%; min $61,089,612
Iter 128 - Loss: -1.988
acq_func took 1.2s
api took 16.1s
time step 2, drop -7.56%; min $61,626,916
Iter 128 - Loss: -1.972
acq_func took 0.2s
api took 14.9s
time step 3, drop -42.35%; min $81,557,688
Iter 128 - Loss: -0.509
acq_func took 2.0s
api took 14.8s
time step 4, drop -0.37%; min $57,504,900
Iter 128 - Loss: -0.680
acq_func took 2.6s
api took 25.3s
time step 5, drop 3.47%; min $55,303,652
Iter 128 - Loss: -0.787
acq_func took 3.1s
api took 15.1s
time step 6, drop 2.71%; min $55,742,448
Iter 128 - Loss: -0.878
acq_func took 2.3s
api took 15.7s
time step 7, drop -9.76%; min $62,886,724
Iter 128 - Loss: -0.875
acq_func took 2.2s
api took 15.2s
time step 8, drop 0.59%; min $56,953,212
Iter 128 - Loss: -0.942
acq_func took 2.3s
api took 15.5s
time step 9, drop 1.29%; min $56,552,992
Iter 128 - Loss: -1.022
acq_func took 2.6s
api took 13.9s
time step 10, drop 2.40%; min $55,917,312
acq_

##### multiple runs